In [1]:
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
import graphlearning as gl
from sklearn.metrics.pairwise import pairwise_distances
import numpy as np
from scipy import sparse, linalg, stats
from scipy.sparse.linalg import svds, aslinearoperator, LinearOperator
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.manifold import _utils
from sklearn.manifold import _t_sne
from sklearn.neighbors import NearestNeighbors
from time import time

In [2]:
n = 20000
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target
y = y.astype(int)
y = np.array(y)
X = np.array(X)
pca = PCA(n_components=64)
X = pca.fit_transform(X)
X = X[:n,:]
y = y[:n]

In [ ]:
# graph construction
desired_perplexity = 30.0
n_neighbors = int(2)
knn = NearestNeighbors(n_neighbors=n_neighbors, metric="euclidean")
knn.fit(X) # <---biggest bottleneck
distances_nn = knn.kneighbors_graph(X, mode="distance").astype(np.float32)
verbose = 0
P_matrix = _t_sne._joint_probabilities_nn(distances_nn, desired_perplexity, verbose) # <---super fast

In [ ]:
# find eigenfunctions (second biggest bottleneck)
number_of_parameters = 200
P = gl.graph(P_matrix)
vals, vecs = P.eigen_decomp(k = number_of_parameters)

In [ ]:
# find out the percent of faulty connections/optional
distances, indices = knn.kneighbors(X)
n_samples = X.shape[0]
proportions = []
for i in range(n_samples):
    neighbor_indices = indices[i,:]
    same_label_count = sum(y[neighbor_indices] == y[i])
    proportions.append(same_label_count / n_neighbors)


average_proportion = np.mean(proportions)
print(f"Average proportion of neighbors with the same label: {average_proportion:.4f}")

In [ ]:
A = np.zeros((number_of_parameters,2))
A[0,0] = 1
A[1,1] = 1
h = 100 # <--- still need gigantic step size

In [ ]:
# faster one, use of random landmarks, possibility for improvement

def repulsive_force_A_with_subsample(A, n, l, a = 0.015):
    l = np.random.choice(n, l, replace=False)
    #rs = Y[l, :]
    Y = vecs[l, :] @ A
    squared_distances = pairwise_distances(Y, metric="euclidean", squared=True)
    repulsive_term = np.clip(1-a*squared_distances**(1/2),0,None)
    np.fill_diagonal(repulsive_term, 0)
    repulsive_sum = repulsive_term.sum()
    distances = np.sqrt(squared_distances)
    S2 = np.zeros_like(distances)
    nonzero_mask = distances > 0
    valid_mask = nonzero_mask & (distances < 1 / a)
    S2[valid_mask] = a / distances[valid_mask] 
    #S2 *= knn_matrix
    #repulsive_sum = 1
    
    grad = (2.0/repulsive_sum) * (S2 @ Y - S2.sum(axis=1).reshape(-1,1) * Y)
    #grad = grad.ravel()
    # can replace this multipy with n**2
    return vecs[l, :].T@grad

In [ ]:
# slower but adaptive bandwidths, should be capable of significant speedup

def vectorized_binary_search_entropy(targets, low, high, Y, tol=1e-6, max_iter=100):
    low, high = np.array(low), np.array(high)  
    for _ in range(max_iter):
        mid = (low + high) / 2  
        entropy_values = entropy_Q(mid, Y) 
        errors = np.abs(entropy_values - targets) 
        mask = entropy_values > targets 
        low = np.where(mask, low, mid) 
        high = np.where(mask, mid, high) 
        if np.all(errors < tol): 
            break
    return mid


def entropy_Q(a, Y):
    squared_distances = pairwise_distances(Y, metric="euclidean", squared=True)
    Q = np.clip(1 - a[:, np.newaxis] * squared_distances ** (1 / 2), 0, None)
    np.fill_diagonal(Q, 0)
    return np.sum(Q * np.log(1e-128 + Q), axis=1)


def find_a(Y, neighbors):    
    targets = -neighbors*np.ones(Y.shape[0])
    low = np.zeros_like(targets) * 0.000001
    high = np.ones_like(targets) * 1000
    a = vectorized_binary_search_entropy(targets, low, high, Y)
    
    return a

def repulsive_force_A_with_subsample_and_adapt(A, n, l, neighbors = 20):
    l = np.random.choice(n, l, replace=False)
    #rs = Y[l, :]
    Y = vecs[l, :] @ A
    a = find_a(Y, neighbors)
    squared_distances = pairwise_distances(Y, metric="euclidean", squared=True)
    repulsive_term = np.clip(1 - a[:, np.newaxis] * squared_distances ** (1 / 2), 0, None)
    np.fill_diagonal(repulsive_term, 0)
    repulsive_sum = repulsive_term.sum()
    distances = np.sqrt(squared_distances)
    S2 = np.zeros_like(distances)
    nonzero_mask = distances > 0
    np.fill_diagonal(distances, 1)
    valid_mask = nonzero_mask & (distances < 1 / a[:, np.newaxis])
    S2[valid_mask] = (a[:, np.newaxis] / distances)[valid_mask]
    #S2 *= knn_matrix
    #repulsive_sum = 1
    
    grad = (2.0) * (S2 @ Y - S2.sum(axis=1).reshape(-1,1) * Y)
    #grad = grad.ravel()
    # can replace this multipy with n**2
    return vecs[l, :].T@grad,a

In [ ]:
# iterations (maybe tricks like momentum can help)
l = 100
for i in range(10000):
    A -=    300*h * np.diag(vals)@A + 10000*h*repulsive_force_A_with_subsample(A, n,l, 0.025)
Y = vecs@A

plt.figure(figsize=(12, 12))
plt.scatter(Y[:, 0], Y[:, 1], c=y, cmap='tab10', s=20)
plt.show()